# Amazon Route53 Health Check
- Route 53에서 설정한 리소스(웹 어플리케이션, 서버 등)의 상태를 모니터링 하는 기능
    - Routing Policy 에서 활용, Amazon CloudWatch 경보 설정하여 알람 처리 가능
- Latency 정보 확인 가능
    - TCP Connection 연결 수립까지 걸린 시간
    - HTTP(S) First Byte를 받기까지 걸린 시간
        - SSl/TLS Handshake 까지 걸린 시간
- CloudWatch Metric으로 Health Check 상태 기록 / 알람 가능
    - US-East-1 리전 전용 (모든 메트릭이 US-East-1 리전으로 모인다)

## Health Check 모니터링 대상
- 특정 리소스
- 다른 헬스체크
- 아마존 클라우드워치 경보
- Route 53 Application Recovery Controller

### 헬스 체크 모니터링 대상 1 - 리소스
- IP주소, 도메인에 주기적으로 요청을 보내 응답 여부를 확인하여 가용성을 확인
- 상태 확인 방법
    - 전 세계의 다수의 헬스체커에서 정해진 프로토콜/주기로 요청을 보내서 상태 확인
    - 주기 : 10초(추가요금) or 30초(체커 별 싱크 없음 - 여러 체커들의 주기 동기화가 안돼있다는 뜻)
- 두 가지 지표를 기준으로 상태 판단
    - 응답 속도(HTTP 연결 수립 : 4초, TCP : 10초, HTTP String Match : 2초)
    - 응답 내용 및 지정한 실패 회수를 연속으로 넘었는지 여부로 상태 판단
- 해당 기준으로 전체 헬스 체커의 18% 초과가 Healthy 상태를 유지해야 Health 상태로 판단
- 주의 : AWS 바깥의 엔드포인트를 대상으로 할 경우 추가 요금 발생 (온프레미스 환경이나 다른 클라우드 프로바이더 체크 가능)

#### 헬스 체크 모니터링 대상 - 리소스 지정 가능 값
- 모드
    - IP 주소 : IPv4, IPv6(로컬/프라비잇/멀티캐스트 등은 체크 불가능)
        - HTTP, HTTPs -> Status 2xx, 3xx
        - TCP
    - 도메인
        - HTTP/HTTPS -> Status 2xx, 3xx
        - IPv4만 지원, 즉 A record 외에는 FAIL 처리
- 매칭 문자열 : 응답의 바디에 특정 문자열이 있는지 확인 (상태가 헬시해야만 OK 응답이 나오도록 만들고, OK 문자열을 확인하도록)
- Health Check Region : 최소 3개
- 기타 : 포트, 경로, 주기, SNI지원, Latency Graph, Inverse Check 등등

## 헬스 체크 모니터링 대상 2 - 다른 헬스 체크
- 다른 헬스 체크를 모니터
    - 예: 3개 이상의 설정한 헬스체크의 상태검사가 실패한 경우 경보/FailOver 등
- 헬스 체크 성공 여부를 정하기 위한 다른 헬스 체크 성공 회수 기준 설정 가능
    - 모든 헬스 체크 성공해야 : 성공
    - 단 하나의 헬스 체크라도 성공해야 : 성공
    - 지정한 헬스 체크 숫자 중 N개 이상이면 : 성공

## 헬스 체크 모니터링 대상 3 - CloudWatch 경보
- 클라우드워치 경보 상태를 모니터링
- 주의
    - 클라우드워치 경보의 경보 상태가 아닌 데이터를 직접 모니터링 : 클라우드워치 경보보다 조금 더 민감하게 반응
    - Standard Resolution(60초마다 수집) 경보만 모니터링 가능
    - Average, Minimum, Maximum, Sum, SampleCount만 모니터링 가능
    - Math Metric 사용 불가능
    - CloudWatch 경보가 변경되었을 경우 Route53 Health Check 수동으로 업데이트 필요
- 데이터가 충분하지 않을 경우(Insufficient Data 상태) 상태 지정 가능
    - 예 : Insufficient Data 일 때 - Healthy, Unhealthy, Last Known Status로 지정 가능

### 데모 - 리소스 헬스체크 모니터링
- EC2 인스턴스를 프로비전하고 라우트53 헬스 체크 구성
    - 인스턴스를 두 개 만들어서 헬스체크를 구성하고, 이 두 헬스체크를 모아서 모니터링하는 헬스체크 생성
- 헬스 체크 실패 시 SNS를 통해 이메일 받아보기

### 실습 순서
1. 인스턴스 두개 프로비전
    - 키페어 없음
    - 보안그룹 디폴트
    - 유저 데이터에 스크립트 입력
2. 라우트53 콘솔로 이동
    - 상태 검사 : 상태 검사 생성
        - 이름 : demo-my-ec2-health-check-1
        - 모니터링 대상 : 엔드포인트
        - 도메인 이름 선택, 인스턴스의 DNS 주소 입력(인스턴스 구분 쉽게 넘버링 해놓자)
        - 경로 : index.html
        - 고급 구성 : 빠름, 실패 임계값 : 1, 문자열 검색 : awsclassroom, 지연 시간 그래프 체크
        - 경보 생성 : 예, 새로운 SNS 주제 : my-route53-health-check-email, 메일 주소 입력
    - demo-my-ec2-health-check-2 도 똑같이 생성
    - demo-healthcheck-all 생성
        - 다른 상태 검사 상태
        - 전체 상태 검사가 정상 체크
        - 경보도 기본 주제 선택
3. 헬스체크의 모니터링으로 들어가보자
    - 인스턴스에도 연결해보자
        - sudo -s
        - tail -f /var/log/httpd/access_log
    - 헬스 체크를 하나 강제로 FAIL 시켜보자
        - 인스턴스 연결, nano /var/www/html/index.html
            - 문자열을 지워보자 : HealthCheck Fail 나면서 메일 올거임
4. 리소스 정리 : 인스턴스, 상태검사 삭제